In [1]:
def add(*args, **kwargs):
    return

In [2]:
import parser
import ast

import numpy as np
import pandas as pd

In [3]:
add(1, 2+3, 999, (i for i in range(10)), **{'a': [1,2,3]})

In [4]:
s = In[3]
s

"add(1, 2+3, 999, (i for i in range(10)), **{'a': [1,2,3]})"

In [11]:
a = ast.parse(s, mode='eval')
# a = ast.parse(s)
a

In [12]:
b = a.body
b

In [19]:
function_name = b.func.id
function_name

'add'

In [20]:
is_ = []
for arg in b.args:
    print(arg.col_offset)
    i = arg.col_offset
    is_.append(i)
    print()
    
for kwarg in b.keywords:
    print(kwarg.value.col_offset)
    i = kwarg.value.col_offset
    is_.append(i)
    print()

4

7

12

18

43



In [21]:
ast.dump(a)

"Expression(body=Call(func=Name(id='add', ctx=Load()), args=[Num(n=1), BinOp(left=Num(n=2), op=Add(), right=Num(n=3)), Num(n=999), GeneratorExp(elt=Name(id='i', ctx=Load()), generators=[comprehension(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Num(n=10)], keywords=[]), ifs=[], is_async=0)])], keywords=[keyword(arg=None, value=Dict(keys=[Str(s='a')], values=[List(elts=[Num(n=1), Num(n=2), Num(n=3)], ctx=Load())]))]))"